In [ ]:
import collections
Individual = collections.namedtuple('Individual', ['Name', 'Marks', 'WebAssign', 'RawData'])
Marks = collections.namedtuple('Marks', ['Score', 'NumberOfNs'])

## Key      |   Data
## ---------+---------
## Name     |  full name:  last, first
## Marks    |  a namedtuple of type Marks
## WebAssign|  WebAssign score (whole number, max 100)
## Raw Data |  full pandas Series whose index is the Name above


The threshold variables are html tables which specify the required thresholds for getting a particular letter grade.

If the grading thresholds change (in a future semester, perhaps), adjust these tables in addition to the code.

In [ ]:
midterm_243 = ("(12.1-12.4 Chapter 6, Chapter 7, 8.1, 8.2)", 
               20, 80,  #A 
               18, 2, 70, #B
               14, 4, 60, #C
               12, 50,  #D
               12, 50) #F

final_243 = ("(all)", 
               44, 80,  #A 
               37, 2, 70, #B
               33, 6, 60, #C
               29, 50,  #D
               29, 50) #F

midterm_242 = ("(through section 3.5)", 
               21, 80,  #A 
               19, 2, 70, #B
               16, 4, 60, #C
               14, 50,  #D
               14, 50) #F

final_242 = ("(all)", 
               43, 80,  #A 
               36, 2, 70, #B
               32, 6, 60, #C
               28, 50,  #D
               28, 50) #F


midterm_321 = ( 5, 3, 2, #A 
               4, 1, 2, 2, #B
               3, 2, 1, 3, #C
               3, 3, #D
               3) #F

final_321 = ( 10, 7, 6, #A 
               7, 3, 5, 8, #B
               5, 5, 5, 5, #C
               5, 7, #D
               5) #F

calculus_threshold_table = """<table class='paleGrayRows'> <thead> <tr> 
<th>Letter Grade</th> 
<th>Course Learning Targets</th> 
<th>WebAssign Homework %s</th> </tr> 
</thead> 
<tbody> 
<tr> <td>A</td> <td>≥ %d points and no scores below proficient</td> <td>≥ %d%%</td> </tr> 
<tr> <td>B</td> <td>≥ %d points and at most %d scores below proficient</td> <td>≥ %d%%</td> </tr> 
<tr> <td>C</td> <td>≥ %d points and at most %d scores below proficient</td> <td>≥ %d%%</td> </tr> 
<tr> <td>D</td> <td>≥ %d points</td> <td>≥ %d%%</td> </tr> 
<tr> <td>F</td> <td>&lt; %d points</td> <td>&lt; %d%%</td> </tr> 
</tbody> </table>"""


discrete_threshold_table = """<table class='paleGrayRows'> <thead> <tr> 
<th>Letter Grade</th> 
<th>Core Learning Targets</th> 
<th>Supplemental Targets</th> </tr> 
</thead> 
<tbody> 
<tr> <td>A</td> <td>Master %d </td> <td>Master %d, Proficient %d</td> </tr> 
<tr> <td>B</td> <td>Master %d, Proficient %d</td>Master %d, Proficient %d <td></td> </tr> 
<tr> <td>C</td> <td>Master %d, Proficient %d</td> <td>Master %d, Proficient %d</td> </tr> 
<tr> <td>D</td> <td>Proficient %d</td> <td>Proficient %d</td> </tr> 
<tr> <td>F</td> <td>Proficient &lt; %d</td> <td>----</td> </tr> 
</tbody> </table>"""

# print(discrete_threshold_table % final_321)

def decide_letter_grade(individual):
    print(threshold_table)
    threshold = { "A": {"Points": threshold_table[1], "Below P": 0, "WebAssign": threshold_table[2]},
           "B": {"Points": threshold_table[3], "Below P": threshold_table[4], "WebAssign": threshold_table[5]},
           "C": {"Points": threshold_table[6], "Below P": threshold_table[7], "WebAssign": threshold_table[8]},
           "D": {"Points": threshold_table[9], "Below P": 1000, "WebAssign": threshold_table[10]},  
           "F": {"Points": threshold_table[11], "Below P": 1000, "WebAssign": 0}
           # yes, I'm using 1000 as a threshold. I'm a teacher, not a programmer...
          }
    
    if individual.Marks.Score >= threshold['A']["Points"] and individual.Marks.NumberOfNs <= threshold["A"]["Below P"] and individual.WebAssign >= threshold["A"]["WebAssign"]:
        return "A"
    elif individual.Marks.Score >= threshold['B']["Points"] and individual.Marks.NumberOfNs <= threshold["B"]["Below P"] and individual.WebAssign >= threshold["B"]["WebAssign"]:
        return "B"
    elif individual.Marks.Score >= threshold['C']["Points"] and individual.Marks.NumberOfNs <= threshold["C"]["Below P"] and individual.WebAssign >= threshold["C"]["WebAssign"]:
        return "C"
    elif individual.Marks.Score >= threshold['D']["Points"] and individual.Marks.NumberOfNs <= threshold["D"]["Below P"] and individual.WebAssign >= threshold["D"]["WebAssign"]:
        return "D"
    elif individual.Marks.Score >= threshold['F']["Points"] and individual.Marks.NumberOfNs <= threshold["F"]["Below P"] and individual.WebAssign >= threshold["F"]["WebAssign"]:
        return "F"
    else:
        return "None of the above?!?"


Here, we feed our data into an HTML template and use *weasyprint* to convert the HTML into a pretty PDF.

In [ ]:
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
from datetime import date

def print_the_report_for(individual):
    today = date.today().strftime("%B %d, %Y")
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("template.html")
    template_vars = {
        "Name": individual.Name,
        "raw_data": individual.RawData.to_html(index=False, bold_rows=False, classes=["paleGrayRows"]),
        "score": individual.Marks.Score,
        "number_of_ns": individual.Marks.NumberOfNs,
        "webassign_grade": individual.WebAssign,
        "letter_grade": decide_letter_grade(individual),
        "date": today,
        "midterm_or_final": midtermOrFinal,
        "grade_threshold": threshold_html_table,
        "extra_text": extra_text if extra_text else ""
    }

    print(individual)
    output = template.render(template_vars)
    HTML(string=output).write_pdf("./reports/"+individual.Name+".pdf",stylesheets=["./report.css"])
   

Read in the data from the Blackboard exported CSV. 


**Future note** if blackboard columns change, this will be rather broken.

If you're just playing with this, you can read *test.csv* for test data.

In [ ]:
import pandas as pd

filename_change_if_youre_testing = "math242_testdata.csv" if course=="math242" else "math243_testdata.csv"

df = pd.read_csv(filename_change_if_youre_testing)

math243_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 
               'V1', 'V2', 'V3', 'V4', 'V5',
               'F1', 'F2', 'F3', 'F4', 
               'INT1', 'INT2', 'INT3', 'INT4', 'INT5', 
               'A1', 'A2', 
               'C1', 'C2', 'C3', 'C4', 'C5',
               'S1', 'S2', 'S3', 'S4', 'S5', 
               'P1', 'P2', 'P3', 'WebAssign']

math242_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 
               'F1', 'F2',
               'L1', 'L2', 
               'D1', 'D2', 
                       'DC1', 'DC2', 'DC3,', 'DC4', 'DC5',
                       'DA1', 'DA2', 'DA2', 'DA3', 'DA4', 'DA5', 'DA6',
                       'INT1', 'INT2', 'INT3', 'INT4', 'INT5', 
               'IA1', 'IA2', 'IA3', 'IA4'
               'IF1', 'IF2', 'WebAssign']
math321_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 
                       'O1', 'O2 Core', 'O3', 'O4 Core', 'O5',
                       'L1 Core', 'L2', 'L3', 'L4', 'L5 Core',
                       'N1 Core', 'N2 Core', 'N3', 'N4', 'N5', 'N6',
                       'R1 Core', 'R2', 'R3', 'R4 Core',
                       'C1 Core', 'C2 Core', 'C3', 'C4']


better_cols = math242_better_cols if course=="math242" else math243_better_cols

df.columns = better_cols

## Make a better index column and dump everything not grade-related
df['Name'] = df['Last Name'] + ', ' + df['First Name']
df.drop(['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability'], axis=1, inplace=True)
df.set_index("Name", inplace=True)




Define the tools for reading the P, PM, and empty strings into their corresponding numeric values

In [ ]:
def sgn_to_number(sgn):
#Note that S has the value 1, G has the value 0.5, and N has the value 0; else is 0; 
    if (sgn == "PM"):
        return 2
    elif (sgn == "P"):
        return 1
    else:
        return 0
    
def marks_to_score(marks):
    
    # this needs to be a dictionary, not a Marks namedtuple
    return {'score' : sgn_to_number(marks), 
                'number of ns' : 1 if marks=="N" else 0}

def summative(marks):
    score_so_far = 0
    ns_so_far = 0
    for cell in marks:
        try: 
            score_so_far += cell['score']
            ns_so_far += cell['number of ns']
        except:
            pass
    return Marks(Score=score_so_far, NumberOfNs=ns_so_far)

Iterate over the dataset, sum marks, assign webassign grade. 

Note: Iterating over a *pandas* DataFrame is a bad idea for large datasets. If you're a student (not me), learn to do this differently. For this use case of no more that 40 records, I'm fine with performance loss.

In [ ]:
class_scores = list()
for index, row in df.iterrows():
    raw = pd.DataFrame({'Standard':row.index, 'Marks':row.values})
    raw = raw.dropna()
    class_scores.append( Individual(Name=index, 
                                    Marks=summative(row.apply(marks_to_score)), 
                                    WebAssign=row.loc['WebAssign'], 
                                    RawData=raw) )

In [1]:
# # temporary stuff:
# midtermOrFinal = "Final"
# #extra_text = "<h2>Final course grade</h2>"
# #extra_text = "<h2>Last week before final</h2><p> <b>Note:</b> this is prepared the week before the final. There is still one more quiz on one standard; you can get up to 2 points (two S's) or as low as 2 additional N's. Note also the grade thresholds that must be met on the next page.</p> <p> Unhappy with your grade? There's still time to reassess - break week, last week of classes, and during finals week. Additionally, the final exam will cover the second half material. </p> <p>Happy with your grade? Do well on the last quiz and you can skip the final.</p>"
extra_text = "The webassign grade was based on score of all assignments that were due before March 13. Work between March 14 and the present is not reflected on this report<br/><br/>"

# for individual in class_scores: 
#     print_the_report_for(individual)

midtermOrFinal = "Midterm"


course = "math242"  #math242 or math243 as a string
# I'm an idiot an still have this midterm vs final; oh well
threshold_table = midterm_242 if course=="math242" else math_243


threshold_html_table = calculus_threshold_table % threshold_table
#for individual in class_scores:
print_the_report_for(class_scores[29])

NameError: name 'midterm_242' is not defined